# 多重处理效应 - 当选择不止两个时

---

## 学习目标

完成本练习后，你将能够：

1. 理解多重处理 (Multiple Treatments) 与二元处理的区别
2. 掌握多重处理的因果识别条件
3. 实现广义倾向得分 (GPS) 和逆概率加权 (IPW) 
4. 应用 Meta-Learners 于多重处理场景
5. 比较不同处理之间的效应差异

---

## 真实场景：多档位优惠策略

### 场景描述

你是某外卖平台的增长负责人，需要设计新用户首单优惠策略。

**可选方案**：
- T=0：无优惠（对照组）
- T=1：满 20 减 5
- T=2：满 30 减 10
- T=3：满 50 减 20

**问题**：
> 不同用户应该分配到哪个优惠档位？每个档位的增量效应是多少？

### 与二元处理的区别

| 维度 | 二元处理 | 多重处理 |
|------|---------|----------|
| **处理变量** | T ∈ {0, 1} | T ∈ {0, 1, 2, ..., K} |
| **潜在结果** | Y(0), Y(1) | Y(0), Y(1), ..., Y(K) |
| **处理效应** | ATE = E[Y(1) - Y(0)] | K 个对比 |
| **倾向得分** | e(X) = P(T=1|X) | 多项 Logit 或 K 个二元 |

### 生动比喻：点菜

- **二元处理**：「要不要加辣？」
- **多重处理**：「微辣、中辣、特辣、还是变态辣？」

不同人对不同辣度的「承受能力」不同，我们要找到每个人的「最佳辣度」。

---

## 核心概念

### 多重处理的潜在结果框架

对于 K+1 个处理水平 $T \in \{0, 1, ..., K\}$：

- **潜在结果**: $\{Y(0), Y(1), ..., Y(K)\}$
- **观测结果**: $Y = \sum_{k=0}^{K} \mathbb{1}(T=k) \cdot Y(k)$

### 因果效应的定义

**与对照组相比的效应**：
$$\tau_k = E[Y(k) - Y(0)]$$

**两个处理之间的效应差**：
$$\tau_{k,j} = E[Y(k) - Y(j)]$$

### 识别假设

1. **无混淆 (No Confounding)**：
   $$Y(k) \perp T | X, \quad \forall k$$

2. **重叠 (Overlap)**：
   $$0 < P(T=k | X) < 1, \quad \forall k, X$$

3. **SUTVA**：
   - 无干扰：一个人的处理不影响另一个人的结果
   - 处理一致性：同一处理对同一人效果相同

---

## 广义倾向得分 (GPS)

### 定义

对于多重处理，倾向得分变成**向量**：

$$\mathbf{e}(X) = [P(T=0|X), P(T=1|X), ..., P(T=K|X)]$$

### 估计方法

1. **多项 Logistic 回归 (Multinomial Logit)**

$$P(T=k|X) = \frac{\exp(\beta_k^T X)}{\sum_{j=0}^{K} \exp(\beta_j^T X)}$$

2. **K 个二元分类器 (One-vs-Rest)**

$$P(T=k|X) \approx \frac{\hat{p}_k(X)}{\sum_{j=0}^{K} \hat{p}_j(X)}$$

---

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict
from typing import Tuple, Dict, List
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0',
}

print("环境准备完成！")

---

## Part 1: 数据生成

In [ ]:
def generate_multi_treatment_data(
    n_samples: int = 5000,
    n_treatments: int = 4,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    生成多重处理数据
    
    场景：外卖平台优惠策略
    - X: 用户特征
    - T: 优惠档位 (0=无, 1=满20减5, 2=满30减10, 3=满50减20)
    - Y: 首单消费金额
    """
    np.random.seed(seed)
    
    # 用户特征
    X = np.random.randn(n_samples, 5)
    # X[:, 0]: 历史消费能力
    # X[:, 1]: 价格敏感度
    # X[:, 2]: 使用频率
    # X[:, 3]: 设备类型 (高端 vs 低端)
    # X[:, 4]: 距离餐厅远近
    
    # 非随机分配：基于用户特征
    # 高消费能力用户更可能收到大额优惠（业务策略）
    logits = np.zeros((n_samples, n_treatments))
    logits[:, 0] = 0  # 基准
    logits[:, 1] = 0.5 + 0.3 * X[:, 0] + 0.2 * X[:, 1]
    logits[:, 2] = 0.3 + 0.5 * X[:, 0] + 0.3 * X[:, 1]
    logits[:, 3] = 0.1 + 0.8 * X[:, 0] + 0.4 * X[:, 1]
    
    # Softmax
    exp_logits = np.exp(logits)
    probs = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    
    # 抽样处理
    T = np.array([np.random.choice(n_treatments, p=p) for p in probs])
    
    # 真实潜在结果
    noise = np.random.randn(n_samples) * 5
    
    # Y(0): 无优惠
    Y0 = 30 + 10 * X[:, 0] + 5 * X[:, 2] + noise
    
    # Y(1): 满20减5，效应取决于价格敏感度
    tau1 = 8 + 3 * X[:, 1]  # 价格敏感的人响应更大
    Y1 = Y0 + tau1
    
    # Y(2): 满30减10
    tau2 = 15 + 5 * X[:, 1] - 2 * X[:, 0]  # 低消费能力人群效应更大
    Y2 = Y0 + tau2
    
    # Y(3): 满50减20
    tau3 = 25 + 8 * X[:, 1] - 5 * X[:, 0]  # 边际效应更大
    Y3 = Y0 + tau3
    
    # 真实潜在结果矩阵
    Y_potential = np.column_stack([Y0, Y1, Y2, Y3])
    
    # 观测结果
    Y = np.array([Y_potential[i, T[i]] for i in range(n_samples)])
    
    # 真实 ATE
    true_ate = {
        'tau_1': np.mean(tau1),
        'tau_2': np.mean(tau2),
        'tau_3': np.mean(tau3),
    }
    
    return X, T, Y, Y_potential, true_ate, probs


# 生成数据
X, T, Y, Y_potential, true_ate, true_probs = generate_multi_treatment_data()

print("数据生成完成！")
print(f"样本数: {len(X)}")
print(f"\n各处理组分布:")
for k in range(4):
    print(f"  T={k}: {(T==k).sum()} ({(T==k).mean():.1%})")

print(f"\n真实 ATE (vs 对照组):")
for k, v in true_ate.items():
    print(f"  {k}: {v:.2f}")

---

## Part 2: 广义倾向得分估计

In [ ]:
class GeneralizedPropensityScore:
    """
    广义倾向得分估计
    
    使用多项 Logistic 回归估计 P(T=k|X)
    """
    
    def __init__(self, method: str = 'multinomial'):
        """
        Args:
            method: 'multinomial' 或 'ovr' (one-vs-rest)
        """
        self.method = method
        self.model = None
        self.n_treatments = None
    
    def fit(self, X: np.ndarray, T: np.ndarray):
        """
        拟合 GPS 模型
        """
        self.n_treatments = len(np.unique(T))
        
        if self.method == 'multinomial':
            self.model = LogisticRegression(
                multi_class='multinomial',
                solver='lbfgs',
                max_iter=1000
            )
        else:
            self.model = LogisticRegression(
                multi_class='ovr',
                solver='lbfgs',
                max_iter=1000
            )
        
        self.model.fit(X, T)
        return self
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        预测各处理的概率
        
        Returns:
            probs: 形状 (n_samples, n_treatments)
        """
        return self.model.predict_proba(X)
    
    def get_propensity(self, X: np.ndarray, T: np.ndarray) -> np.ndarray:
        """
        获取每个样本实际接受的处理的倾向得分
        
        Returns:
            ps: 形状 (n_samples,)
        """
        probs = self.predict_proba(X)
        ps = np.array([probs[i, T[i]] for i in range(len(T))])
        return ps


# 估计 GPS
gps_model = GeneralizedPropensityScore(method='multinomial')
gps_model.fit(X, T)

# 预测概率
pred_probs = gps_model.predict_proba(X)

print("GPS 估计完成！")
print(f"\n预测概率分布（前 5 个样本）:")
print(pd.DataFrame(pred_probs[:5], columns=[f'P(T={k})' for k in range(4)]).round(3))

In [ ]:
# 检查 GPS 估计质量
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('真实 vs 预测概率', '倾向得分分布'))

# 真实 vs 预测
for k in range(4):
    fig.add_trace(go.Scatter(
        x=true_probs[:100, k],
        y=pred_probs[:100, k],
        mode='markers',
        name=f'T={k}',
        marker=dict(size=5, opacity=0.5)
    ), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(dash='dash', color='gray'),
    name='理想线',
    showlegend=False
), row=1, col=1)

# 倾向得分分布
ps = gps_model.get_propensity(X, T)
for k in range(4):
    mask = T == k
    fig.add_trace(go.Histogram(
        x=ps[mask],
        name=f'T={k}',
        opacity=0.5,
        nbinsx=30
    ), row=1, col=2)

fig.update_xaxes(title_text='真实概率', row=1, col=1)
fig.update_yaxes(title_text='预测概率', row=1, col=1)
fig.update_xaxes(title_text='倾向得分', row=1, col=2)
fig.update_yaxes(title_text='频数', row=1, col=2)

fig.update_layout(height=400, template='plotly_white',
                  title_text='广义倾向得分估计质量')
fig.show()

---

## Part 3: 多重处理 IPW 估计

In [ ]:
class MultiTreatmentIPW:
    """
    多重处理逆概率加权估计
    
    估计 E[Y(k)] = E[Y * I(T=k) / P(T=k|X)]
    """
    
    def __init__(self, trim_quantile: float = 0.01):
        """
        Args:
            trim_quantile: 权重裁剪分位数
        """
        self.trim_quantile = trim_quantile
        self.gps = None
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        拟合 IPW 估计器
        """
        self.X = X
        self.T = T
        self.Y = Y
        self.n_treatments = len(np.unique(T))
        
        # 估计 GPS
        self.gps = GeneralizedPropensityScore()
        self.gps.fit(X, T)
        
        return self
    
    def estimate_ate(self, reference: int = 0) -> Dict:
        """
        估计相对于参照组的 ATE
        
        Args:
            reference: 参照处理水平
        
        Returns:
            各处理水平的 ATE 估计
        """
        probs = self.gps.predict_proba(self.X)
        
        results = {}
        mu_estimates = {}
        
        # 估计每个处理水平的均值
        for k in range(self.n_treatments):
            mask = self.T == k
            
            # 获取权重
            weights = 1 / probs[mask, k]
            
            # 裁剪极端权重
            lower = np.quantile(weights, self.trim_quantile)
            upper = np.quantile(weights, 1 - self.trim_quantile)
            weights = np.clip(weights, lower, upper)
            
            # 归一化权重
            weights = weights / weights.sum() * mask.sum()
            
            # 加权均值
            mu_estimates[k] = np.average(self.Y[mask], weights=weights)
        
        # 计算 ATE
        for k in range(self.n_treatments):
            if k != reference:
                results[f'tau_{k}'] = mu_estimates[k] - mu_estimates[reference]
        
        results['mu_estimates'] = mu_estimates
        
        return results


# IPW 估计
ipw_estimator = MultiTreatmentIPW(trim_quantile=0.01)
ipw_estimator.fit(X, T, Y)
ipw_results = ipw_estimator.estimate_ate(reference=0)

print("IPW 估计结果")
print("="*60)
print(f"\n处理水平均值估计:")
for k, mu in ipw_results['mu_estimates'].items():
    print(f"  E[Y({k})]: {mu:.2f}")

print(f"\nATE 估计 (vs 对照组):")
print(f"{'处理':<10} {'IPW估计':<15} {'真实值':<15} {'误差':<10}")
print("-"*50)
for k in [1, 2, 3]:
    ipw_est = ipw_results[f'tau_{k}']
    true_val = true_ate[f'tau_{k}']
    error = ipw_est - true_val
    print(f"tau_{k:<7} {ipw_est:<15.2f} {true_val:<15.2f} {error:<10.2f}")

---

## Part 4: 多重处理 Meta-Learners

### T-Learner 的多处理扩展

In [ ]:
class MultiTreatmentTLearner:
    """
    多重处理 T-Learner
    
    为每个处理水平训练独立的结果模型
    """
    
    def __init__(self, base_model=None):
        if base_model is None:
            base_model = GradientBoostingRegressor(n_estimators=100, max_depth=4)
        self.base_model = base_model
        self.models = {}
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        为每个处理水平训练模型
        """
        self.n_treatments = len(np.unique(T))
        
        for k in range(self.n_treatments):
            mask = T == k
            
            # 克隆基模型
            from sklearn.base import clone
            model = clone(self.base_model)
            
            # 训练
            model.fit(X[mask], Y[mask])
            self.models[k] = model
        
        return self
    
    def predict_potential_outcomes(self, X: np.ndarray) -> np.ndarray:
        """
        预测所有潜在结果
        
        Returns:
            Y_pred: 形状 (n_samples, n_treatments)
        """
        Y_pred = np.zeros((len(X), self.n_treatments))
        
        for k in range(self.n_treatments):
            Y_pred[:, k] = self.models[k].predict(X)
        
        return Y_pred
    
    def estimate_cate(self, X: np.ndarray, reference: int = 0) -> Dict:
        """
        估计 CATE
        """
        Y_pred = self.predict_potential_outcomes(X)
        
        cate = {}
        for k in range(self.n_treatments):
            if k != reference:
                cate[f'tau_{k}'] = Y_pred[:, k] - Y_pred[:, reference]
        
        return cate
    
    def estimate_ate(self, X: np.ndarray, reference: int = 0) -> Dict:
        """
        估计 ATE
        """
        cate = self.estimate_cate(X, reference)
        
        ate = {}
        for key, values in cate.items():
            ate[key] = np.mean(values)
        
        return ate


# T-Learner 估计
t_learner = MultiTreatmentTLearner()
t_learner.fit(X, T, Y)
t_learner_ate = t_learner.estimate_ate(X, reference=0)

print("T-Learner 估计结果")
print("="*60)
print(f"\nATE 估计 (vs 对照组):")
print(f"{'处理':<10} {'T-Learner估计':<15} {'真实值':<15} {'误差':<10}")
print("-"*50)
for k in [1, 2, 3]:
    t_est = t_learner_ate[f'tau_{k}']
    true_val = true_ate[f'tau_{k}']
    error = t_est - true_val
    print(f"tau_{k:<7} {t_est:<15.2f} {true_val:<15.2f} {error:<10.2f}")

### 双重稳健估计 (AIPW)

In [ ]:
class MultiTreatmentAIPW:
    """
    多重处理的增强逆概率加权 (AIPW)
    
    结合 IPW 和结果回归，双重稳健
    """
    
    def __init__(self, trim_quantile: float = 0.01):
        self.trim_quantile = trim_quantile
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        self.X = X
        self.T = T
        self.Y = Y
        self.n_treatments = len(np.unique(T))
        
        # 估计 GPS
        self.gps = GeneralizedPropensityScore()
        self.gps.fit(X, T)
        
        # 估计结果模型
        self.outcome_models = {}
        for k in range(self.n_treatments):
            mask = T == k
            model = GradientBoostingRegressor(n_estimators=100, max_depth=4)
            model.fit(X[mask], Y[mask])
            self.outcome_models[k] = model
        
        return self
    
    def estimate_ate(self, reference: int = 0) -> Dict:
        """
        AIPW 估计
        
        公式:
        E[Y(k)] = (1/n) * Σ [μ_k(X) + I(T=k)/e_k(X) * (Y - μ_k(X))]
        """
        n = len(self.X)
        probs = self.gps.predict_proba(self.X)
        
        mu_estimates = {}
        
        for k in range(self.n_treatments):
            # 结果模型预测
            mu_k = self.outcome_models[k].predict(self.X)
            
            # 倾向得分
            e_k = probs[:, k]
            e_k = np.clip(e_k, 0.01, 0.99)  # 裁剪
            
            # 指示函数
            indicator = (self.T == k).astype(float)
            
            # AIPW 估计
            aipw_terms = mu_k + indicator / e_k * (self.Y - mu_k)
            mu_estimates[k] = np.mean(aipw_terms)
        
        # 计算 ATE
        results = {}
        for k in range(self.n_treatments):
            if k != reference:
                results[f'tau_{k}'] = mu_estimates[k] - mu_estimates[reference]
        
        results['mu_estimates'] = mu_estimates
        
        return results


# AIPW 估计
aipw_estimator = MultiTreatmentAIPW()
aipw_estimator.fit(X, T, Y)
aipw_results = aipw_estimator.estimate_ate(reference=0)

print("AIPW 估计结果")
print("="*60)
print(f"\nATE 估计 (vs 对照组):")
print(f"{'处理':<10} {'AIPW估计':<15} {'真实值':<15} {'误差':<10}")
print("-"*50)
for k in [1, 2, 3]:
    aipw_est = aipw_results[f'tau_{k}']
    true_val = true_ate[f'tau_{k}']
    error = aipw_est - true_val
    print(f"tau_{k:<7} {aipw_est:<15.2f} {true_val:<15.2f} {error:<10.2f}")

---

## Part 5: 方法对比与可视化

In [ ]:
# 汇总所有方法的估计结果
comparison = pd.DataFrame({
    '真实值': [true_ate['tau_1'], true_ate['tau_2'], true_ate['tau_3']],
    'IPW': [ipw_results['tau_1'], ipw_results['tau_2'], ipw_results['tau_3']],
    'T-Learner': [t_learner_ate['tau_1'], t_learner_ate['tau_2'], t_learner_ate['tau_3']],
    'AIPW': [aipw_results['tau_1'], aipw_results['tau_2'], aipw_results['tau_3']],
}, index=['满20减5', '满30减10', '满50减20'])

print("方法对比")
print("="*60)
print(comparison.round(2).to_string())

# 可视化
fig = go.Figure()

treatments = ['满20减5', '满30减10', '满50减20']
methods = ['真实值', 'IPW', 'T-Learner', 'AIPW']
colors = [COLORS['success'], COLORS['primary'], COLORS['warning'], COLORS['danger']]

for method, color in zip(methods, colors):
    fig.add_trace(go.Bar(
        name=method,
        x=treatments,
        y=comparison[method].values,
        marker_color=color
    ))

fig.update_layout(
    title='多重处理 ATE 估计对比',
    xaxis_title='优惠档位',
    yaxis_title='ATE (相对于无优惠)',
    barmode='group',
    template='plotly_white',
    height=450
)
fig.show()

---

## Part 6: 异质效应分析

In [ ]:
# 分析不同用户群体的 CATE
cate_all = t_learner.estimate_cate(X, reference=0)

# 按价格敏感度分组
sensitivity_groups = pd.cut(X[:, 1], bins=3, labels=['低敏感', '中敏感', '高敏感'])

cate_by_group = pd.DataFrame({
    '价格敏感度': sensitivity_groups,
    'CATE_1': cate_all['tau_1'],
    'CATE_2': cate_all['tau_2'],
    'CATE_3': cate_all['tau_3'],
})

group_means = cate_by_group.groupby('价格敏感度').mean()

print("不同价格敏感度用户的 CATE")
print("="*60)
print(group_means.round(2).to_string())

# 可视化
fig = go.Figure()

for col in ['CATE_1', 'CATE_2', 'CATE_3']:
    fig.add_trace(go.Bar(
        name=col.replace('CATE_', '方案'),
        x=group_means.index.tolist(),
        y=group_means[col].values
    ))

fig.update_layout(
    title='不同价格敏感度用户的 CATE',
    xaxis_title='价格敏感度',
    yaxis_title='CATE',
    barmode='group',
    template='plotly_white',
    height=400
)
fig.show()

print("""
📊 观察:
1. 高价格敏感度用户对各档位优惠的响应都更大
2. 大额优惠（满50减20）对低敏感度用户效果有限
3. 这提示我们应该给高敏感度用户发大额优惠，给低敏感度用户发小额优惠
""")

---

## 思考题

### 基础理解

1. **多重处理和连续处理有什么区别？** 什么时候用哪种方法？

2. **为什么多重处理需要更强的重叠假设？** 如果某个处理水平的样本很少怎么办？

### 深入分析

3. **如何处理「处理之间存在层级关系」的情况？** 例如，满50减20 包含了 满30减10 的优惠。

4. **AIPW 为什么是「双重稳健」的？** 如果 GPS 或结果模型有一个是错误的，会发生什么？

### 实战应用

5. **如何为每个用户选择最优的处理水平？** 需要考虑哪些因素（成本、风险等）？

6. **如果有预算约束，如何在用户群体中分配不同档位的优惠？**

---

## 总结

| 概念 | 定义 | 重要性 |
|------|------|--------|
| **多重处理** | T ∈ {0, 1, ..., K} | 更真实的业务场景 |
| **广义倾向得分** | P(T=k\|X) 向量 | 识别的基础 |
| **多处理 IPW** | 加权估计 E[Y(k)] | 经典方法 |
| **多处理 AIPW** | 结合 IPW 和 OR | 双重稳健 |
| **最优处理选择** | argmax_k CATE_k | 个性化策略 |

### 方法对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| IPW | 简单 | 方差大 | 大样本 |
| T-Learner | 灵活 | 无法处理稀疏 | 各组样本充足 |
| AIPW | 双重稳健 | 计算复杂 | 通用 |

---

*恭喜你完成了多重处理效应的学习！* 🎉